Chroma DB ile Semantik Arama 

In [1]:
import chromadb

import pandas as pd

/Users/gizemkaryagdi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/y6/npmgkj1j4mzgx950yrf632lc0000gn/T/ipykernel_6278/1351659426.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df=pd.read_csv("medium_post_titles.csv")
df=df.dropna() #dropna:dataframe(df) ten alınan tüm null değerlerini kaldırır.
df=df[~df["subtitle_truncated_flag"]]
topics_of_interest=['artificial-intelligence','data-science','machine-learning']
df=df[df['category'].isin(topics_of_interest)]
df['text']=df['title']+df['subtitle']
df['meta']=df.apply(lambda x:{
    'text':x['text'],
    'category':x['category']
}, axis=1)

In [3]:
df.head(2)

,category,title,subtitle,subtitle_truncated_flag,text,meta
4,artificial-intelligence,"""Can I Train my Model on Your Computer?""",How we waste computational resources and how t...,False,"""Can I Train my Model on Your Computer?""How we...","{'text': '""Can I Train my Model on Your Comput..."
289,data-science,(Robot) data scientists as a service,Automating data science with symbolic regressi...,False,(Robot) data scientists as a serviceAutomating...,{'text': '(Robot) data scientists as a service...


Chroma DB Setup
Veri eklemek için vektör yerleştirmelerinin yapılması gerekiyor. Ancak vektör veri tabanına veri eklemeden önce bir koleksiyona sahip olunması gerekir bu nedenle bir koleksiyon oluşturulmalı koleksiyon: Temel olarak vektörleri kimlikleri ve diğer bilgileri tanımladığımız bir vektör bütünüdür. Bu bir nevi "indeks" gibidir. Kısaca "Dizin" gibi.

In [4]:
chroma_client=chromadb.Client()
article_collection=chroma_client.create_collection(name="medium-article")

VEKTÖR VERİLERİNİ EKLEME 
'upsert' kullanımı mevcut olan bir vektör kimliği bulursa bunun tekrar eklenmemesini sağlar. Yani güncellemeniz gereken herhangi bir vektör varsa kullanılır.
'add' ilk kez eklenecek bir vektör bulunuyorsa kullanılır.

In [5]:
article_collection.upsert(
    ids=[f"{x}" for x in df.index.tolist()],
    documents=df['text'].tolist(),
    metadatas=df['meta'].tolist()
)

2024-02-07 18:08:51.292987 [W:onnxruntime:, helper.cc:67 IsInputSupported] CoreML does not support input dim > 16384. Input:embeddings.word_embeddings.weight, shape: {30522,384}
2024-02-07 18:08:51.293465 [W:onnxruntime:, coreml_execution_provider.cc:81 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 49 number of nodes in the graph: 323 number of nodes supported by CoreML: 231


VEKTÖR SORGUSU


In [6]:
qry_str="best ai library?"

In [7]:
article_collection.query(query_texts=qry_str,n_results=2)

{'ids': [['103719', '112075']],
 'distances': [[0.6766335368156433, 0.7713300585746765]],
 'metadatas': [[{'category': 'machine-learning',
    'text': 'Top 7 libraries and packages of the year for Data Science and AI: Python & RThis is a list of the best libraries and packages that changed our lives this year, compiled from my weekly digests'},
   {'category': 'machine-learning',
    'text': 'What are Some ‘Advanced ‘ AI and Machine Learning Online Courses?Where can you find advanced AI and machine learning courses? A comprehensive review based on my personal experience with these courses.'}]],
 'embeddings': None,
 'documents': [['Top 7 libraries and packages of the year for Data Science and AI: Python & RThis is a list of the best libraries and packages that changed our lives this year, compiled from my weekly digests',
   'What are Some ‘Advanced ‘ AI and Machine Learning Online Courses?Where can you find advanced AI and machine learning courses? A comprehensive review based on my p